<a href="https://colab.research.google.com/github/hebaasim/Drowsinessdetection/blob/main/real_timedriverdrowsinessdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
# Install necessary dependencies
!pip install torch torchvision numpy opencv-python
!git clone https://github.com/ultralytics/yolov5  # YOLOv5 repo
!pip install -r yolov5/requirements.txt
!pip install simpleaudio  # Install simpleaudio for playing sounds

  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83
Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.12 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (11858/11858), done.
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia

In [ ]:
import torch
import cv2
import numpy as np
import simpleaudio as sa
import matplotlib.pyplot as plt
from IPython.display import display, Audio, Javascript
from google.colab.output import eval_js
from base64 import b64decode

ModuleNotFoundError: No module named 'simpleaudio'

In [ ]:
img="https://ultralytics.com/images/zidane.jpg"

In [ ]:
model=torch.hub.load('ultralytics/yolov5', 'yolov5s')
results= model (img)

results.print()
%matplotlib inline

plt.imshow(np.squeeze(results.render()))

plt.show()

In [ ]:




# Load the Haar cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Function to capture video frames from the webcam in Colab
def capture_frame_colab():
    js = Javascript('''
        async function captureFrame() {
            const video = document.createElement('video');
            video.style.display = 'none';  // Hide video element
            document.body.appendChild(video);
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            const ctx = canvas.getContext('2d');

            while (true) {
                ctx.drawImage(video, 0, 0);
                const data = canvas.toDataURL('image/jpeg');
                google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
                if (data) {
                    stream.getTracks().forEach(track => track.stop());
                    video.remove();
                    return data;
                }
            }
        }
        captureFrame();
    ''')
    display(js)
    data = eval_js('captureFrame()')
    binary = b64decode(data.split(',')[1])
    return np.frombuffer(binary, dtype=np.uint8)

# Load YOLOv5 model
model = torch.hub.load('yolov5', 'custom', path='yolov5s.pt', source='local')

# Define the function to detect drowsiness
def detect_drowsiness(frame, threshold=1):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    drowsy = False
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(face_roi)

        # Check if eyes are closed (fewer than 2 eyes detected)
        if len(eyes) < 2:
            drowsy = True
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    return drowsy, frame

# Main loop for real-time frame capture and detection
try:
    print("Starting real-time drowsiness detection...")
    while True:
        print("Capturing frame from webcam...")
        frame_data = capture_frame_colab()
        frame = cv2.imdecode(frame_data, cv2.IMREAD_COLOR)
        if frame is None:
            print("Failed to capture frame.")
            break

        # Resize frame for faster processing
        resized_frame = cv2.resize(frame, (640, 480))
        drowsy, annotated_frame = detect_drowsiness(resized_frame)

        if drowsy:
            print("Drowsiness detected! Triggering alarm...")
            try:
                # Play the alarm using IPython's Audio function
                display(Audio('alarm.wav', autoplay=True))
            except Exception as alarm_error:
                print(f"Error playing alarm: {alarm_error}")
        else:
            print("Driver is alert.")

        # Display the frame using matplotlib
        plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')  # Hide axes
        plt.show()

        # Pause for a moment to prevent excessive CPU usage
        cv2.waitKey(1)  # Not strictly needed in Colab

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    plt.close()
